# Ćwiczenie 3 Łączenie z bazą danych i instrukcja select

Zaimportowanie potrzebnych bibliotek oraz połączenie z bazą danych

In [1]:
import psycopg2 as pg
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

# Zadanie 1

In [3]:
cat_num = pd.read_sql('SELECT COUNT(*) FROM category', con=connection)
print(cat_num)

   count
0     16


Ilość kategorii filmów, które znajdują się w wypożyczalni, to 16.

# Zadanie 2

In [4]:
category = pd.read_sql('SELECT name FROM category ORDER BY name ASC', con=connection)
print(category)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


Lista kategorii w kolejności alfabetycznej jest widoczna powyżej.

# Zadanie 3

In [8]:
youngest = pd.read_sql('SELECT title FROM film WHERE release_year = (SELECT MIN(release_year) FROM film) LIMIT 1', con=connection)
print("Najmłodszy film:")
print(youngest)

oldest = pd.read_sql('SELECT title FROM film WHERE release_year = (SELECT MAX(release_year) FROM film) LIMIT 1', con=connection)
print("\nNajstarszy film:")
print(oldest)

Najmłodszy film:
             title
0  Chamber Italian

Najstarszy film:
             title
0  Chamber Italian


Powyższa instrukcja wskazuje na ten sam film będący zarówno najstarszym jak i najmłodszym. Wynika to z faktu, iż wszystkie filmy mają ten sam rok wydania, w związku z czym nie da się ich posortować ze względu na datę.

# Zadanie 4

In [10]:
between_1 = pd.read_sql("SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2005-07-01'::timestamp AND '2005-08-01'::timestamp", con=connection)
print(between_1)

   count
0   6709


Między 2005-07-01 a 2005-08-01 odbyło się 6709 wypożyczeń.

# Zadanie 5

In [9]:
between_2 = pd.read_sql("SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2010-01-01'::timestamp AND '2011-02-01'::timestamp", con=connection)
print(between_2)

   count
0      0


Między 2010-01-01 a 2011-02-01 nie odbyło się żadne wypożyczenie.

# Zadanie 6

In [11]:
max_pay = pd.read_sql('SELECT MAX(amount) FROM payment', con=connection)
print(max_pay)

     max
0  11.99


Największa płatność wypożyczenia wyniosła 11.99.

# Zadanie 7

In [14]:
pol_nig_ban = pd.read_sql("SELECT customer_id, first_name, last_name FROM customer WHERE address_id IN \
                            (SELECT address_id FROM address WHERE city_id IN \
                              (SELECT city_id FROM city WHERE country_id IN \
                                (SELECT country_id FROM country WHERE country IN('Poland', 'Nigeria', 'Bangladesh'))))", con=connection)

print(pol_nig_ban)

    customer_id first_name  last_name
0            18      Carol     Garcia
1            21   Michelle      Clark
2            80    Marilyn       Ross
3           103     Gladys   Hamilton
4           128   Marjorie     Tucker
5           156     Bertha   Ferguson
6           198      Elsie     Kelley
7           232  Constance       Reid
8           250         Jo     Fowler
9           270       Leah     Curtis
10          277       Olga    Jimenez
11          280     Tracey    Barrett
12          284      Sonia    Gregory
13          286      Velma      Lucas
14          318      Brian      Wyman
15          329      Frank   Waggoner
16          332    Stephen     Qualls
17          380    Russell    Brinson
18          409     Rodney    Moeller
19          501      Ruben      Geary
20          558     Jimmie  Eggleston
21          562    Wallace      Slone
22          571    Johnnie   Chisholm
23          572     Sidney   Burleson


Powyżej widoczni są wszyscy klienci z Polski, Nigerii oraz Bangladeszu.

# Zadanie 8

In [30]:
staff = pd.read_sql("SELECT country.country, city.city, address.address FROM country, city, address, staff \
                      WHERE country.country_id = city.country_id \
                      AND city.city_id = address.city_id \
                      AND address.address_id = staff.address_id", con=connection)

print(staff)

     country        city               address
0     Canada  Lethbridge     23 Workhaven Lane
1  Australia   Woodridge  1411 Lillydale Drive


Członkowie personelu mieszkają w Kanadzie oraz Australii, a ich dokładne adresy są widoczne powyżej.

# Zadanie 9

In [32]:
arg_or_spain = pd.read_sql("SELECT COUNT(*) FROM staff WHERE address_id IN \
                            (SELECT address_id FROM address WHERE city_id IN \
                              (SELECT city_id FROM city WHERE country_id IN \
                                (SELECT country_id FROM country WHERE country IN('Argentina', 'Spain'))))", con=connection)

print(arg_or_spain)

   count
0      0


W Argentynie lub w Hiszpanii nie mieszka żaden z pracowników.

# Zadanie 10

In [34]:
cat_rent = pd.read_sql("SELECT name FROM category WHERE category_id IN \
                        (SELECT category_id FROM film_category WHERE film_id IN \
                          (SELECT film_id FROM film WHERE film_id IN \
                            (SELECT film_id FROM inventory WHERE inventory_id IN \
                              (SELECT inventory_id FROM rental))))", con=connection)

print(cat_rent)

           name
0        Family
1        Travel
2        Sports
3      Classics
4        Action
5           New
6        Comedy
7        Horror
8      Children
9        Sci-Fi
10        Music
11        Games
12      Foreign
13  Documentary
14    Animation
15        Drama


Na tej liście znajdują się wszystkie kategorie filmów wypożyczanych przez klientów.

# Zadanie 11

In [35]:
film_usa = pd.read_sql("SELECT name FROM category, film_category, film, inventory, rental, staff, address, city, country, store \
                        WHERE category.category_id = film_category.category_id \
                        AND film_category.film_id = film.film_id \
                        AND film.film_id = inventory.film_id \
                        AND inventory.inventory_id = rental.inventory_id \
                        AND rental.staff_id = staff.staff_id \
                        AND staff.address_id = address.address_id \
                        AND address.city_id = city.city_id \
                        AND city.country_id = country.country_id \
                        AND country.country = 'United States' \
                        AND staff.store_id = store.store_id \
                        AND store.address_id = address.address_id", con=connection)

print(film_usa)

Empty DataFrame
Columns: [name]
Index: []


Jak widać w Ameryce nie został wypożyczone żadne kategorie filmów, ponieważ dostaliśmy pusty DataFrame.

# Zadanie 12

In [39]:
act = pd.read_sql("SELECT title FROM film WHERE film_id IN \
                    (SELECT film_id FROM film_actor WHERE actor_id IN \
                      (SELECT actor_id FROM actor WHERE \
                        first_name = 'Olympia' AND last_name = 'Pfeiffer' OR \
                        first_name = 'Julia' AND last_name = 'Zellweger' OR \
                        first_name = 'Ellen' AND last_name = 'Presley'))", con=connection)

print(act)

                title
0   Caribbean Liberty
1      Roots Remember
2        Outlaw Hanky
3        Women Dorado
4    Empire Malkovich
..                ...
64  Contact Anonymous
65    Majestic Floats
66       Deep Crusade
67   Effect Gladiator
68     Express Lonely

[69 rows x 1 columns]


Tytuły filmów, które zostały wyświetlone to filmy, w których grał: Olympia Pfeiffer, Julia Zellweger lub Ellen Presley.